In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
data = {
    "date": ["10/17", "10/18", "10/19"],
    "9am": [90, 100, 111],
    "10am": [95, 105, 112],
    "2pm": [99, 110, 113]
}
df = pd.DataFrame(data)
df

,date,9am,10am,2pm
0,10/17,90,95,99
1,10/18,100,105,110
2,10/19,111,112,113


In [3]:
# Sort data by date
df = df.sort_values(by='date')

# Drop 'date' column for now
df = df.drop(['date'], axis=1)

# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
scaled_data

array([[0.        , 0.        , 0.        ],
       [0.47619048, 0.58823529, 0.78571429],
       [1.        , 1.        , 1.        ]])

In [4]:
# Initialize X and y using the scaled data
X, y = [], []

# Column indices in scaled_data corresponding to '2pm', '9am', '10am'
idx_2pm, idx_9am, idx_10am = df.columns.get_loc('2pm'), df.columns.get_loc('9am'), df.columns.get_loc('10am')

for i in range(1, len(scaled_data)):
    X.append([scaled_data[i-1, idx_2pm], scaled_data[i, idx_9am], scaled_data[i, idx_10am]])
    y.append(scaled_data[i, idx_2pm])

X

[[0.0, 0.4761904761904763, 0.5882352941176476], [0.7857142857142856, 1.0, 1.0]]

In [5]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Create the GRU model
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(3, 1)))
model.add(GRU(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [7]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.5611
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.5153
Epoch 3/100
1/1 [==============================] - 0s 8ms/step - loss: 0.4710
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 0.4277
Epoch 5/100
1/1 [==============================] - 0s 8ms/step - loss: 0.3855
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 0.3442
Epoch 7/100
1/1 [==============================] - 0s 8ms/step - loss: 0.3039
Epoch 8/100
1/1 [==============================] - 0s 8ms/step - loss: 0.2647
Epoch 9/100
1/1 [==============================] - 0s 8ms/step - loss: 0.2266
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 0.1899
Epoch 11/100
1/1 [==============================] - 0s 11ms/step - loss: 0.1550
Epoch 12/100
1/1 [==============================] - 0s 10ms/step - loss: 0.1222
Epoch 13/100
1/1 [==============================] - 0s 10ms

In [8]:
# Prediction and performance check
predicted_stock_price = model.predict(X_test)

# Inverse transformation to get actual value
predicted_stock_price = scaler.inverse_transform(np.concatenate((X_test.reshape(X_test.shape[0], -1)[:, :-1], predicted_stock_price), axis=1))[:, -1]
predicted_stock_price


1/1 [==============================] - 1s 655ms/step


AttributeError: 'list' object has no attribute 'reshape'